# Load libraries

In [1]:
import numpy as np
import pandas as pd
import os
import ast

import git
from git import RemoteProgress

from git import Repo
from datetime import datetime, date
import matplotlib.pyplot as plt
import seaborn as sns
from time import strptime
%matplotlib inline

In [2]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [3]:
# uncomment the correct repo you want the information from



# # # # apache poi 
# remote_link = "https://github.com/apache/poi"
# local_link = "repo/poi"
# # Uncomment to clone
# # Repo.clone_from(remote_link, local_link, progress=Progress())
# repo = Repo(local_link)
# fixing_commit = "3a328aa220f6979f9805f658ae33244d153beaa7"
# affected_files = ["src/ooxml/java/org/apache/poi/POIXMLTypeLoader.java",
#                   "src/ooxml/java/org/apache/poi/util/DocumentHelper.java"]




# # cloudfoundry uua 
remote_link = "https://github.com/cloudfoundry/uaa"
local_link = "repo/uua"
# Uncomment to clone
Repo.clone_from(remote_link, local_link, progress=Progress())
repo = Repo(local_link)
fixing_commit = "14c745aa293b8d3ce9cdd6bfbc6c0ef3f269b21"
affected_files = ["server/src/main/java/org/cloudfoundry/identity/uaa/zone/GeneralIdentityZoneConfigurationValidator.java"]




# # apache struts 
# remote_link = "https://github.com/apache/struts"
# local_link = "repo/struts"
# # Uncomment to clone
# # Repo.clone_from(remote_link, local_link, progress=Progress())
# repo = Repo(local_link)
# fixing_commit = "74e26830d2849a84729b33497f729e0f033dc147"
# affected_files = ["core/src/main/java/com/opensymphony/xwork2/security/DefaultExcludedPatternsChecker.java",
#                   "core/src/test/java/com/opensymphony/xwork2/interceptor/ParametersInterceptorTest.java"]



remote: Counting objects:   0% (1/1597)
remote: Counting objects:   1% (16/1597)
remote: Counting objects:   2% (32/1597)
remote: Counting objects:   3% (48/1597)
remote: Counting objects:   4% (64/1597)
remote: Counting objects:   5% (80/1597)
remote: Counting objects:   6% (96/1597)
remote: Counting objects:   7% (112/1597)
remote: Counting objects:   8% (128/1597)
remote: Counting objects:   9% (144/1597)
remote: Counting objects:  10% (160/1597)
remote: Counting objects:  11% (176/1597)
remote: Counting objects:  12% (192/1597)
remote: Counting objects:  13% (208/1597)
remote: Counting objects:  14% (224/1597)
remote: Counting objects:  15% (240/1597)
remote: Counting objects:  16% (256/1597)
remote: Counting objects:  17% (272/1597)
remote: Counting objects:  18% (288/1597)
remote: Counting objects:  19% (304/1597)
remote: Counting objects:  20% (320/1597)
remote: Counting objects:  21% (336/1597)
remote: Counting objects:  22% (352/1597)
remote: Counting objects:  23% (368/1597)


Receiving objects:   0% (1/147048)
Receiving objects:   1% (1471/147048), 260.00 KiB | 500.00 KiB/s
Receiving objects:   2% (2941/147048), 260.00 KiB | 500.00 KiB/s
Receiving objects:   3% (4412/147048), 260.00 KiB | 500.00 KiB/s
Receiving objects:   4% (5882/147048), 260.00 KiB | 500.00 KiB/s
Receiving objects:   4% (6175/147048), 260.00 KiB | 500.00 KiB/s
Receiving objects:   5% (7353/147048), 1.48 MiB | 1.46 MiB/s
Receiving objects:   6% (8823/147048), 1.48 MiB | 1.46 MiB/s
Receiving objects:   7% (10294/147048), 3.92 MiB | 2.59 MiB/s
Receiving objects:   8% (11764/147048), 3.92 MiB | 2.59 MiB/s
Receiving objects:   8% (12012/147048), 6.73 MiB | 3.34 MiB/s
Receiving objects:   9% (13235/147048), 6.73 MiB | 3.34 MiB/s
Receiving objects:  10% (14705/147048), 6.73 MiB | 3.34 MiB/s
Receiving objects:  11% (16176/147048), 9.55 MiB | 3.80 MiB/s
Receiving objects:  11% (17059/147048), 9.55 MiB | 3.80 MiB/s
Receiving objects:  12% (17646/147048), 12.36 MiB | 4.10 MiB/s
Receiving objects:  1

Resolving deltas:  39% (24069/61659)
Resolving deltas:  40% (24696/61659)
Resolving deltas:  41% (25356/61659)
Resolving deltas:  42% (25937/61659)
Resolving deltas:  43% (26516/61659)
Resolving deltas:  44% (27136/61659)
Resolving deltas:  45% (27766/61659)
Resolving deltas:  46% (28417/61659)
Resolving deltas:  47% (29006/61659)
Resolving deltas:  48% (29620/61659)
Resolving deltas:  49% (30302/61659)
Resolving deltas:  50% (30838/61659)
Resolving deltas:  51% (31735/61659)
Resolving deltas:  52% (32077/61659)
Resolving deltas:  53% (32693/61659)
Resolving deltas:  54% (33328/61659)
Resolving deltas:  55% (33984/61659)
Resolving deltas:  56% (34550/61659)
Resolving deltas:  57% (35149/61659)
Resolving deltas:  58% (35816/61659)
Resolving deltas:  59% (36390/61659)
Resolving deltas:  60% (37004/61659)
Resolving deltas:  61% (37716/61659)
Resolving deltas:  62% (38233/61659)
Resolving deltas:  63% (38904/61659)
Resolving deltas:  64% (39467/61659)
Resolving deltas:  65% (40080/61659)
R

# Helper functions

In [4]:
# given the line @@ -20,0 +21 @@ import java.io.File; it will return the additions and deletions from it e.g. additions = 1, deletions = 0
def get_additions_deletions_values(line):
    if line[0] == "@@":
        
        # get the values for the upcomming lines e.g. -21,5 +20,5
        deletions = line[1]
        additions = line[2]
        
        # remove symbol (i.e. + or -)
        deletions = deletions[1:]
        additions = additions[1:]
        
        deletions = deletions.split(",")
        additions = additions.split(",")
        
        #if the additions has a "+21,0" means no addiitons
        if len(additions) < 2:
            additions.append("1")
                
        #if the deletions has a "-25,0" means no addiitons
        if len(deletions) < 2:
            deletions.append("1")
            
        return additions, deletions

In [5]:
# finds the code block of the affected line "{ lines affected }"
def find_start_end_of_affected_line(file, line_affected):
    result = []
    start = -1
    end = -1
    classifier = "None"
    current_line = 1
    
    for line in file:
        
        if current_line == line_affected:
            
            i = current_line
            found = False
            
            # find the start block
            while not found:
                i = i - 1
                
                # if cqant fin a "{"
                if i == -1:
                    break
            
                #  if they find the "{" means it is the start
                if "{" in file[i - 1]:
                    found = True
                    start = i
                
                    break;
                
                
            found = False
            i = current_line
            
            # find the end block
            while not found:
                i = i + 1
                
                # if cant fin a "{"
                if i >= (len(file) - 1):
                    break
                
                # if they find the "}" means it is the end
                if "}" in file[i - 1]:
                    found = True
                    end = i
                    break   
        current_line = current_line + 1
    return start, end   

In [6]:
# adds the value to the input list
def add_to_list(input_list, value):
    found = False
    for item in input_list:
        if item["commit"] == value:
            item["count"] = item["count"] + 1
            found = True
    if not found:
        input_list.append({"commit": value, "count": 1})
    return input_list

In [7]:
# find the most common commits found, can exclude a commit (i.e. exclude the current commit number)
def most_common(commits_found, exclude):
    maximum = -1
    commit_found = None
    for commit in commits_found:
        if exclude in commit["commit"]:
            continue
        if commit["count"] > maximum:
            commit_found = commit["commit"]
            maximum = commit["count"]
            
    return commit_found, maximum
        
        

In [8]:
# returns the vcc for a list of files
def get_vcc(files, parameters=None):
    vcc = {}
    for file in files:

        # returns the changes one line at a time
        diff_of_file = repo.git.diff(fixing_commit + "^" + ".." + fixing_commit, "-U0", "--", file).splitlines()

        # reomve unwated stuff
        diff_of_file = diff_of_file[4:]

        # stores the file
        raw_commit_file = repo.git.show(fixing_commit + ":" + file).splitlines()

        # store all the commits found
        commits_found = []

        i = 0
        while i < len(diff_of_file):
            split_line = diff_of_file[i].split()
            if split_line[0] == "@@":
                additions, deletions = get_additions_deletions_values(split_line)

                # if there are deletions loop through them, section 3a
                if deletions[1] != 0:
                    j = 0
                    line_number = int(deletions[0])
                    # loop through the deletions
                    while j < int(deletions[1]):
                        j = j + 1
                        affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(line_number) + "," + str(line_number), fixing_commit, "--", file).splitlines()
                        # print(affected_file_diff)
                        split_line = affected_file_diff[0].split()
                        add_to_list(commits_found, split_line[0])
                    i = i + j

                # if there are additons loop through them section 3b
                if additions[1] != 0:
                    j = 0
                    line_number = int(additions[0])
                    # loop through the additions
                    while j < int(additions[1]):
                        j = j + 1
                        # find the code block that contains the affected line (start line and end line)            
                        start, end = find_start_end_of_affected_line(raw_commit_file, j + line_number)
                        # if it found the start and end line                         
                        if start != -1 and end != -1:
                            affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(start) + "," + str(end), fixing_commit, "--", file).splitlines()
                        # if there was no code block                         
                        else:
                            affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(line_number) + "," + str(line_number), fixing_commit, "--", file).splitlines()
                        
                        # for each line found, add it to the commits list                         
                        for line in affected_file_diff:
                                split_line = line.split()
                                add_to_list(commits_found, split_line[0])
                        line_number = line_number + 1
                        
                    # skip over the lines for the additions
                    i = i + j
            # go to the next line                 
            i = i + 1
            
    return most_common(commits_found, fixing_commit)
#     return vcc

# Main code block

In [9]:
# code block which finds the results and prints them

# paramters to add to the git blame call
parameters = None
# parameters = "-w"
# parameters = "-wM"
# parameters = "-wC"
# parameters = "-wCC"
# parameters = "-wCCC"

# returns the vcc's for each file
vccs = get_vcc(affected_files, parameters)

print("vcc_commit:", vccs[0])
print("amount:", vccs[1])


vcc_commit: 8dff169699d44c602c7b5942ba790e197117d627
amount: 10
